# 诗歌生成

# 数据处理

In [109]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets

start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r',encoding='utf-8') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
    print(examples)      
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))# 分批次
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

# 模型代码， 完成建模代码

In [110]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    batch_input_shape=[None, None])
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids):
        '''
        此处完成建模过程，可以参考Learn2Carry
        '''
        embed_inp=self.embed_layer(inp_ids)
        rnn_out=self.rnn_layer(embed_inp)
        logits=self.dense(rnn_out)
        return logits
    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
    
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [111]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [112]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型
    '''
    # 对于每个批次，我们打开一个 GradientTape() 作用域
    # 在此作用域内，我们调用模型（前向传递）并计算损失
    # 在作用域之外，我们检索模型权重相对于损失的梯度
    # 最后，我们根据梯度使用优化器来更新模型的权重
    with tf.GradientTape() as tape:
        logits=model(x)
        loss=compute_loss(logits,y,seqlen)
    # 计算梯度
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [113]:
tf.config.list_physical_devices('GPU')
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(200):
    loss = train(epoch, model, optimizer, train_ds)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



epoch 0 : loss 8.821194
epoch 1 : loss 6.5866814
epoch 2 : loss 6.1631026
epoch 3 : loss 5.913829
epoch 4 : loss 5.782124
epoch 5 : loss 5.5399804
epoch 6 : loss 5.5113435
epoch 7 : loss 5.381382
epoch 8 : loss 5.3222694
epoch 9 : loss 5.2432833
epoch 10 : loss 5.2163587
epoch 11 : loss 5.1238117
epoch 12 : loss 5.144847
epoch 13 : loss 5.083682
epoch 14 : loss 5.050141
epoch 15 : loss 5.0235844
epoch 16 : loss 4.940289
epoch 17 : loss 4.9784665
epoch 18 : loss 4.973465
epoch 19 : loss 4.896656
epoch 20 : loss 4.841279
epoch 21 : loss 4.842565
epoch 22 : loss 4.824851
epoch 23 : loss 4.796395
epoch 24 : loss 4.7195573
epoch 25 : loss 4.8104196
epoch 26 : loss 4.8434944
epoch 27 : loss 4.6526713
epoch 28 : loss 4.723554
epoch 29 : loss 4.7344084
epoch 30 : loss 4.6919703
epoch 31 : loss 4.6582923
epoch 32 : loss 4.722633
epoch 33 : loss 4.6937575
epoch 34 : loss 4.65296
epoch 35 : loss 4.699846


KeyboardInterrupt: 

# 生成过程

In [114]:
def gen_sentence():
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id['bos']], dtype=tf.int32)
    collect = []
    for _ in range(50):
        cur_token, state = model.get_next_token(cur_token, state)
        collect.append(cur_token.numpy()[0])
    return [id2word[t] for t in collect]
print(''.join(gen_sentence()))

玉楼看玉镜，春色满枝枝。eos得无人事，何人不可怜。eos花不可见，不见白云中。eos鸟来时见，青山不可寻。eos花


In [115]:
def gen_sentence(start_word):
    # 初始化状态
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id[start_word]], dtype=tf.int32)
    collect = [start_word]

    # 生成诗句
    for _ in range(12):  # 限制生成长度为50个词
        # 获取下一个词
        cur_token, state = model.get_next_token(cur_token, state)
        token_id = cur_token.numpy()[0]
        word = id2word[token_id]

        # 添加到诗句中
        collect.append(word)

    return ''.join(collect)

# 生成以特定词语开头的诗句
start_words = ['日', '红', '山', '夜', '湖', '海', '月']
for start_word in start_words:
    print(gen_sentence(start_word))

日暮暮江湖上山。eos不能，不
红粉暖红尘。eos花不可见，妾
山中事，不见王孙。eos为三献
夜，夜来飞鸟啼猿啼。eos花不
湖畔草，山水日暮云。eos云不
海棠幢下不禁，万里闻清风。
月，南山下兮无人间。eos生不
